Geometric model of "S. Stefano Rotondo al Celio", Rome. Model of "Gerusalemme Celeste"
==============================================

In [1]:
from pyplasm import *
from math import sin, cos, pi

#-- 1 cubito = 444 mm ----------------------------------------#
r0 = 24. #---cubiti raggio interno del muro interno------#
r2 = 48. #---cubiti raggio esterno del muro medio--------#
r3 = 72. #---cubiti raggio esterno del muro esterno------#
r4 = 88. #---cubiti raggio esterno scale esterne---------#

wsteps = 10.  #---cubiti profondita' scale esterne-------#
nsteps = 10.  #---numero alzate/pedate scale esterne------#
wstep = wsteps/nsteps #---cubiti larghezza pedata-------#
hstep = 1./3 #---cubiti altezza alzata-------------------#
hw4 = 29. #--cubiti altezza all'imposta del muro esterno-#
hbasament = hstep*nsteps


Evaluating fenvs.py..
...fenvs.py imported in 0.005458 seconds


In [2]:
#Cylindric map of an input domain
cylMap = MAP([lambda p: (p[1]) * math.sin(p[0]), lambda p: (p[1]) * math.cos(p[0]), lambda p: (p[2])])

# Stairs

In [3]:
#External stairway
def vdom(h):
    return PROD([COMP([EMBED(1),INTERVALS(PI*3/24)])(3),QUOTE([h])])

def hdom(w):
    return COMP([S(2)(-1), EMBED(1)])(PROD([INTERVALS(PI*3/24)(3), QUOTE([w])]))

def steps(w, h):
    def steps0(n):
        return COMP([STRUCT, CAT, N(n)])([vdom(h), T(3)(h), hdom(w), T(2)(-w)])
    return steps0

hbasament = hstep * nsteps

# stair + ramp -> stairs
stair = RIGHT([COMP([MKPOL, UKPOL])(steps(wstep, hstep)(nsteps)),COMP([S(2)(-1), SKEL_2, CUBOID])([PI/24, wsteps, hbasament])])
ramp = cylMap(T(2)(r4)(stair))

In [4]:
stairs = STRUCT(NN(12)([ramp, R([1, 2])(PI/6)]))
VIEW(stairs)

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x105049060> >

### note:
press < t > to see it properly!

![stairs_1](./images/stairs_1.png)

![stairs_2](./images/stairs_2.png)

# Basis and basement

In [5]:
basisSector = COMP([cylMap, EMBED(1)])(PROD([INTERVALS(2 * PI / 12)(4), INTERVALS(r4 - (wstep * (nsteps - 1)))(1)]))
basis = COMP([STRUCT, NN(12)])([basisSector, R([1, 2])(2 * PI / 12)])
basement = COMP([R([1, 2])(PI / -48), STRUCT])([stairs, T(3)(hstep * nsteps), basis])

In [6]:
VIEW(basement)

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x10a567060> >

![basement](./images/basement.png)

# Walls

In [7]:
#We are going to use points in the 2D space then the cartesian product
ExtWall2Da = INTERSECTION([MKPOL([[[0, 0], [7, 0], [7, 5], [0, 8], [7, 3], [9, 4], [10.5, 1.5], [10.5, 0], [11, 1.5], [11, 3]],
    [range(1, 5), [2, 8, 7, 6, 5], [6, 7, 9, 10]], [[1], [2], [3]]]), PROD([COMP([QUOTE, N(12)])(1), Q(8)])])

In [8]:
ExtWall2Db = INTERSECTION([MKPOL([[[0.5, 0], [4, 0], [5, 0], [6, 0], [6, 3], [5.5, 3.5], [5, 3], [4.5, 3.5], [4, 3], [2, 4], [0, 3], [0, 1.5], [0.5, 1.5]], [[10, 11, 12, 13], [1, 2, 9, 10, 13], [2, 3, 7, 8, 9], [3, 4, 5, 6, 7]],range(1, 5)]),
    PROD([COMP([QUOTE, N(6)])(1), Q(4)])])


In [9]:
ExtWall2D = STRUCT([ExtWall2Da, T(1)(11), ExtWall2Db])
sizxExtWall = SIZE(1)(ExtWall2D)
ExtWall = R([2, 3])(PI/2)(PROD([ExtWall2D, QUOTE([1.5])]))

In [10]:
CurvedExtWall = COMP([cylMap, T(2)(r3), S([1, 3])([PI/(4*sizxExtWall), hw4/8])])(ExtWall)
DoubleExtWall = STRUCT([CurvedExtWall, S(1)(-1), CurvedExtWall])
FullExtWall = COMP([STRUCT, NN(4)])([DoubleExtWall, R([1, 2])(PI/2)])

In [11]:
VIEW(FullExtWall)

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x10a569b10> >

![extwall_1](./images/extwall_1.png)

![extwall_2](./images/extwall_2.png)

# Columns

In [12]:
hCol = 12 #column height

def Column(args):
    """
    :param w: width
    :param h: height
    :return: pyplasm.xgepy
    """
    w, h = args
    basis = CUBOID([w, w, 2*w/3])
    trunk = CYLINDER([w/2*.85, h-w])(8)
    capitel = CUBOID([w, w, w/3])
    return TOP([TOP([basis, trunk]), capitel])

arcAngle = 2*PI/50.4
wallAngle = -3.2*arcAngle/4.
RotCross = 3.2*arcAngle/4. + 2.5*arcAngle

myColumn = COMP([MKPOL, UKPOL, T(2)(r2-.75), Column])([1.5, hCol])

In [13]:
VIEW(myColumn)

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x10483a660> >

![column](./images/column.png)

In [14]:
the4cols = COMP([R([1, 2])(arcAngle*.4/3.2), MKPOL, UKPOL, STRUCT, NN(4)])([R([1, 2])(DIFF(arcAngle)), myColumn])
the5cols = COMP([R([1, 2])(arcAngle*.4/3.2), MKPOL, UKPOL, STRUCT, NN(5)])([R([1,2])(DIFF(arcAngle)), myColumn])

In [15]:
VIEW(the4cols)
VIEW(the5cols)

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x10483a630> >

![5cols_1](./images/5cols_1.png)

![5cols_2](./images/5cols_2.png)

In [16]:
theBotWal = COMP([cylMap, MKPOL, UKPOL, T(2)(r2-.75), CUBOID])([3.2*arcAngle/4, 1.5, hCol])
theSecCols = STRUCT([R([1, 2])(RotCross), theBotWal, R([1, 2])(wallAngle), the4cols, R([1,2])(-5*arcAngle), theBotWal, R([1,2])(wallAngle), the5cols])
theMedColumns = COMP([STRUCT, NN(4)])([theSecCols, R([1, 2])(PI/-2)])

VIEW(theMedColumns)

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x10483aa50> >

![med columns](./images/medcolumns.png)

![med columns2](./images/medcolumns_2.png)

In [27]:
def bottomArc(d):
    """
    :param d: diameter
    """
    return BEZIER(S1)([[0,0],[0,2*d/3],[d,2*d/3],[d,0]])

def topArc(d):
    return BEZIER(S1)([[0,2*d/3],[d,2*d/3]])

def arc2D(d):
    return BEZIER(S2)([bottomArc(d),topArc(d)])
"""
DEF arc3D  (d::IsRealPos)(w::IsRealPos) =
    (T:2:w ~ R:<2,3>:(PI/2)):
        ( MAP:(CONS:(arc2D:d)):(Intervals:1:8 * Intervals:1:1) * QUOTE:<w> );

def arc3D(d):
    def arc3D1(w):
        return COMP([T(2)(w),R([2,3])(PI/2)])(MAP([CONS([arc2D(d)])])(PROD([PROD([INTERVALS(1)(8),INTERVALS(1)(1)]),QUOTE([w])])))
    return arc3D1
"""
def arc3D(d):
    """
    :param d: diameter
    :return: function __main__.arc3D
    """
    def arc3D1(w):
        """
        :param w: width
        :return: function __main__.arc3D
        """
        arco = arc2D(d)
        dominio = PROD([INTERVALS(1)(8), INTERVALS(1)(1)])
        ar = MAP(arco)(dominio)
        domin = PROD([ar, QUOTE([1.5])])
        return COMP([T(2)(w), R([2, 3])(PI / 2)])(domin)
    return arc3D1

def Interarc(d1, d2):
    def Interarc1(w):
        return CUBOID([d1, w, 2 * d2 / 3])
    return Interarc1

def Xarc(d1, d2):
    def Xarc1(w):
        return RIGHT([RIGHT([Interarc(d1, d2)(w), arc3D(d2)(w)]), Interarc(d1, d2)(w)])
    return Xarc1

theArc = Xarc(0.4, 3.2)(1.5)

sizeArc = SIZE(1)(theArc)
sx = (1 / sizeArc) * arcAngle
sz = sx * (r2 - 2)
curvedArc = COMP([cylMap, MKPOL, UKPOL, T(2)(r2 - .75), S([1, 3])([sx, sz])])(theArc)
heigthArc = SIZE(3)(curvedArc)
the5arcs = COMP([MKPOL, UKPOL, STRUCT, NN(5)])([curvedArc, R([1, 2])(-arcAngle)])
the6arcs = COMP([MKPOL, UKPOL, STRUCT, NN(6)])([curvedArc, R([1, 2])(-arcAngle)])
theMidWall = COMP([cylMap, MKPOL, UKPOL, T(2)(r2-.75), CUBOID])([3.2 * arcAngle/4,1.5, heigthArc])
theSector = STRUCT([R([1, 2])(RotCross),theMidWall, R([1,2])(wallAngle), the5arcs, R([1,2])(-5*arcAngle), theMidWall,R([1,2])(wallAngle), the6arcs])
theArcs = COMP([STRUCT, NN(4)])([theSector, R([1, 2])(PI/-2)])
theArcsCols = STRUCT([theMedColumns, T(3)(hCol), theArcs])


In [28]:
VIEW(theArcs)

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x110ae1e10> >

In [30]:
VIEW(theArcsCols)

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x110ae1f90> >